# Make a coastal sfincs model from scratch using hydromt

This notebook demonstrates how to prepare and inspect a SFINCS model from scratch. The notebook does not contain the SFINCS code or executables to run the model with. 

We will make a schematization with the following features:
* we only simulate coastal flooding, we do not consider any interactions with riverine flooding
* we use the MERIT Hydro dataset as elevation
* we include boundary conditions from the Global Tide and Surge Model by imposing a number of boundary points    where forcing is expected



### HydroMT CLI build interface
First let us see what hydromt provides us with.

In [ ]:
!hydromt --help

It looks like we have several commands that we can consider. We want to `build` a model. How does that work? We can also call `--help` on the available hydromt commands as follows.

In [ ]:
!hydromt build --help

### Building a first model (and getting an error when data is missing)
Let's try to build an entire schematisation, just from the command line (Awesome!!!). In the `--help` section, there is an example for making a model from a bounding box, which is very intuitive. Let's try exactly that. We store the model on the local path `./texel_sfincs`

In [ ]:
!hydromt build sfincs ./texel_sfincs "{'bbox': [4.6891,52.9750,4.9576,53.1994]}"

The above might have taken a few seconds, but then should have miserably crashed! What happened:
* HydroMT was looking for a data catalog to use. 
* It did not find any and therefore, it automatically downloaded a sample data catalog which you can now find on your home folder under .hydromt_data
* It tried to build a model in the Texel bounding box, but...alas, the sample dataset does not cover Texel at all!


### Building a first model (and getting it right)
Let's adapt the bounding box to an area that fits within the sample dataset. The dataset covers the Piave basin and its surroundings in Italy. A good place to get a CSV formatted bounding box is https://boundingbox.klokantech.com/

We also add `-vv` for extra verbosity

In [ ]:
!hydromt build sfincs ./venice_sfincs "{'bbox': [12.047909,45.15809,12.833432,45.646944]}" -vv

This should work without any errors. 

The example above means the following: run **hydromt build** with:

* `sfincs` : i.e. build a sfincs model
* `./venice_sfincs` : output model folder
* `"{'bbox': [12.047909,45.15809,12.833432,45.646944]}"` : make a model that is active for the full given bounding box. All *REGION* options are described in the [docs](https://deltares.github.io/hydromt/latest/user_guide/cli.html#region-options)
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

NOTE: As we did not specify a model configuration, only the base maps (topography) have been setup using default parameters. To build a complete model we need the use a configuraton ini-file.

We can now inspect the newly created folder `./venice_sfincs`

In [ ]:
ls ./venice_sfincs

The .log file contains the same messages as shown in the verbose messages during the building of the model. The files are standard sfincs formats and contain the following
* .inp file : holds the model's configuration
* .dep file : holds the bathymetry in the typical format of sfincs
* .msk file : mask file (0, inactive cells, 1, active cells, 2, boundary cells)

Let us have a closer look at the model's configuration

In [ ]:
fn_inp = './venice_sfincs/sfincs.inp'
with open(fn_inp, 'r') as f:
    txt = f.read()

print(txt)

We see that quite standard settings are used. The coordinate reference system (see `epsg`) is automatically chosen to the nearest UTM zone. We have not even considered the resolution. It is now 100 meters (see `dx` and `dy`). Let's assume we want a 50 meter model and rebuild it. The `--help` section showed that the `-r` option has to be used to change this. Building will take longer.

In [ ]:
!hydromt build sfincs ./venice_sfincs "{'bbox': [12.047909,45.15809,12.833432,45.646944]}" -r 50 -vv
fn_inp = './venice_sfincs/sfincs.inp'
with open(fn_inp, 'r') as f:
    txt = f.read()

print(txt)

It is clear that the `dx` and `dy` settings are changed.

### Inspecting the model
Let's have a closer look at the model. To interact with a model within a notebook or python script, we need to load a number of modules.

In [ ]:
%matplotlib inline
import hydromt
import os


Now we can load the model in memory and inspect it.

In [ ]:
# define the model's configuration file
root = r'./venice_sfincs'
config_fn = os.path.abspath(os.path.join(root, 'sfincs.inp'))
print(config_fn)
# read the model with hydromt sfincs methods
mod = hydromt.SfincsModel(root=root, config_fn=config_fn, mode="r")
mod.read()
mod.plot_basemap(figsize=(12, 10))

### Next steps
You can update an existing model, e.g. to replace parameterisations or add forcing. this is descibed in a follow-up notebook `sfincs_update_model.ipynb`